In [2]:
import os
import matplotlib.pyplot as plt
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
/opt/conda/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [3]:
import torch
import sys
import os
import torch
import numpy
import importlib
import torch.nn.functional as F
from scipy.io import wavfile
from collections import defaultdict

from sklearn import manifold
from torchsummary import summary
import time

from thop import profile

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
sys.path.append('/workspace/LOGS_OUTPUT/server9_nvme1/ASV_LOGS_202102/train_dist')
sys.path.append('/workspace/GREAT_ASV_system/Model_exp')

# SE-Res2net50

In [6]:
SpeakerNetModel = importlib.import_module('models.'+'Res2Net50').__getattribute__('MainModel')

In [7]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=64, nOut=192, spec_aug=False)

Res2Net50.py, Embedding size is 192, Encoder SAP, Spec_aug False.


In [8]:
S.cuda()

ResNetSE(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBottle2neck(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool): AvgPool2d(kernel_size=3, stride=1, padding=1)
      (convs): ModuleList(
        (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bns): ModuleList(
        (0): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [9]:
S.eval()

ResNetSE(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBottle2neck(
      (conv1): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (pool): AvgPool2d(kernel_size=3, stride=1, padding=1)
      (convs): ModuleList(
        (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (bns): ModuleList(
        (0): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

In [10]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [11]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [12]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [13]:
len(setfiles)

4708

In [23]:
testcount = 1000
starttime = time.time()
for count, wavline in enumerate(setfiles[:testcount]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime))
print(testcount / (endtime-starttime))

12.256335020065308
81.59045900449541


In [14]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_avgpool() for <class 'torch.nn.modules.pooling.AvgPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.ModuleList'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.Sigmoid'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'models.ResNetBlocks.SELayer'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'models.ResNetBlocks.SEBottl

In [15]:
flops / 1000000

3277.71568

In [16]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 64, 751]               0
    MelSpectrogram-4              [-1, 64, 751]               0
    InstanceNorm1d-5              [-1, 64, 751]               0
            Conv2d-6          [-1, 32, 64, 751]             320
              ReLU-7          [-1, 32, 64, 751]               0
       BatchNorm2d-8          [-1, 32, 64, 751]              64
            Conv2d-9          [-1, 32, 64, 751]           1,024
      BatchNorm2d-10          [-1, 32, 64, 751]              64
             ReLU-11          [-1, 32, 64, 751]               0
           Conv2d-12           [-1, 8, 64, 751]             576
      BatchNorm2d-13           [-1, 8, 64, 751]              16
             ReLU-14           [-1, 8, 

# RepVGG B0 40d

In [4]:
SpeakerNetModel = importlib.import_module('models.'+'repVGG_deploy').__getattribute__('MainModel')

In [26]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

In [27]:
S.cuda()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(

In [28]:
S.eval()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(

In [29]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [30]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [31]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [32]:
len(setfiles)

4708

In [23]:
testcount = 1000
starttime = time.time()
for count, wavline in enumerate(setfiles[:testcount]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime))
print(testcount / (endtime-starttime))

12.256335020065308
81.59045900449541


In [33]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.RepVGGBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.AttentiveStatsPool'>. Treat it as zero Macs and zero Params.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transf

In [34]:
flops / 1000000

2112.089984

In [35]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv2d-6          [-1, 64, 20, 376]             640
              ReLU-7          [-1, 64, 20, 376]               0
       RepVGGBlock-8          [-1, 64, 20, 376]               0
            Conv2d-9          [-1, 64, 10, 188]          36,928
             ReLU-10          [-1, 64, 10, 188]               0
      RepVGGBlock-11          [-1, 64, 10, 188]               0
           Conv2d-12          [-1, 64, 10, 188]          36,928
             ReLU-13          [-1, 64, 10, 188]               0
      RepVGGBlock-14          [-1, 64, 

# RepVGG B0 64d

In [5]:
SpeakerNetModel = importlib.import_module('models.'+'repVGG_deploy').__getattribute__('MainModel')

In [6]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=64, nOut=192, spec_aug=False)

In [8]:
S.cuda()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(

In [9]:
S.eval()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(

In [10]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [11]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [12]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [13]:
len(setfiles)

4708

In [20]:
testcount = 1000
starttime = time.time()
for count, wavline in enumerate(setfiles[:testcount]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime))
print(testcount / (endtime-starttime))

6.782984972000122
147.4277186412705


In [21]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.RepVGGBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.AttentiveStatsPool'>. Treat it as zero Macs and zero Params.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transf

In [22]:
flops / 1000000

2940.396928

In [23]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 64, 751]               0
    MelSpectrogram-4              [-1, 64, 751]               0
    InstanceNorm1d-5              [-1, 64, 751]               0
            Conv2d-6          [-1, 64, 32, 376]             640
              ReLU-7          [-1, 64, 32, 376]               0
       RepVGGBlock-8          [-1, 64, 32, 376]               0
            Conv2d-9          [-1, 64, 16, 188]          36,928
             ReLU-10          [-1, 64, 16, 188]               0
      RepVGGBlock-11          [-1, 64, 16, 188]               0
           Conv2d-12          [-1, 64, 16, 188]          36,928
             ReLU-13          [-1, 64, 16, 188]               0
      RepVGGBlock-14          [-1, 64, 

# RepVGG B1 40d

In [22]:
SpeakerNetModel = importlib.import_module('models.'+'repVGG_deploy').__getattribute__('MainModel')

In [23]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

In [24]:
S.cuda()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam):

In [25]:
S.eval()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam):

In [26]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [27]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [28]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [29]:
len(setfiles)

4708

In [35]:
testcount = 1000
starttime = time.time()
for count, wavline in enumerate(setfiles[:testcount]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime))
print(testcount / (endtime-starttime))

6.997316360473633
142.91193201564954


In [36]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.RepVGGBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.AttentiveStatsPool'>. Treat it as zero Macs and zero Params.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transf

In [37]:
flops / 1000000

8138.789248

In [38]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv2d-6          [-1, 64, 20, 376]             640
              ReLU-7          [-1, 64, 20, 376]               0
       RepVGGBlock-8          [-1, 64, 20, 376]               0
            Conv2d-9         [-1, 128, 10, 188]          73,856
             ReLU-10         [-1, 128, 10, 188]               0
      RepVGGBlock-11         [-1, 128, 10, 188]               0
           Conv2d-12         [-1, 128, 10, 188]         147,584
             ReLU-13         [-1, 128, 10, 188]               0
      RepVGGBlock-14         [-1, 128, 

# RepVGG B1 64d

In [5]:
SpeakerNetModel = importlib.import_module('models.'+'repVGG_deploy').__getattribute__('MainModel')

In [6]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=64, nOut=192, spec_aug=False)

In [7]:
S.cuda()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam):

In [8]:
S.eval()

RepVGG(
  (stage0): RepVGGBlock(
    (nonlinearity): ReLU()
    (rbr_reparam): Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (stage1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (stage2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (rbr_reparam):

In [9]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [10]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [11]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [12]:
len(setfiles)

4708

In [18]:
testcount = 1000
starttime = time.time()
for count, wavline in enumerate(setfiles[:testcount]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime))
print(testcount / (endtime-starttime))

7.044663190841675
141.95142803988662


In [19]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.RepVGGBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[WARN] Cannot find rule for <class 'models.repVGG_deploy.AttentiveStatsPool'>. Treat it as zero Macs and zero Params.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transf

In [20]:
flops / 1000000

11358.793088

In [21]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 64, 751]               0
    MelSpectrogram-4              [-1, 64, 751]               0
    InstanceNorm1d-5              [-1, 64, 751]               0
            Conv2d-6          [-1, 64, 32, 376]             640
              ReLU-7          [-1, 64, 32, 376]               0
       RepVGGBlock-8          [-1, 64, 32, 376]               0
            Conv2d-9         [-1, 128, 16, 188]          73,856
             ReLU-10         [-1, 128, 16, 188]               0
      RepVGGBlock-11         [-1, 128, 16, 188]               0
           Conv2d-12         [-1, 128, 16, 188]         147,584
             ReLU-13         [-1, 128, 16, 188]               0
      RepVGGBlock-14         [-1, 128, 

# resnet34L

In [37]:
SpeakerNetModel = importlib.import_module('models.'+'ResNet34L_new').__getattribute__('MainModel')

In [38]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(nOut=192, spec_aug=False, n_mels=64)

ResNet34L_new.py, Embedding size is 192, Encoder SAP, Spec_aug False.


In [39]:
S.cuda()

ResNetSE(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, 

In [40]:
S.eval()

ResNetSE(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, 

In [41]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [42]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [43]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [44]:
len(setfiles)

4708

In [94]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)
print(testcount / (endtime-starttime))

0.12144420266151429


In [45]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[WARN] Cannot find rule for <class 'models.ResNetBlocks.BasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.Spectrogram'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelScale'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelSpectrogram'>. Treat it as zero Macs and ze

In [46]:
flops / 1000000

3456.534656

In [47]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 64, 751]               0
    MelSpectrogram-4              [-1, 64, 751]               0
    InstanceNorm1d-5              [-1, 64, 751]               0
            Conv2d-6          [-1, 16, 64, 751]             160
              ReLU-7          [-1, 16, 64, 751]               0
       BatchNorm2d-8          [-1, 16, 64, 751]              32
            Conv2d-9          [-1, 16, 64, 751]           2,304
             ReLU-10          [-1, 16, 64, 751]               0
      BatchNorm2d-11          [-1, 16, 64, 751]              32
           Conv2d-12          [-1, 16, 64, 751]           2,304
      BatchNorm2d-13          [-1, 16, 64, 751]              32
             ReLU-14          [-1, 16, 

# SE-Resnet34L

In [48]:
SpeakerNetModel = importlib.import_module('models.'+'ResNetSE34L_new').__getattribute__('MainModel')

In [49]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(nOut=192, spec_aug=False, n_mels=64)

ResNetSE34L_new.py, Embedding size is 192, Encoder SAP, Spec_aug False.


In [50]:
S.cuda()

ResNetSE(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=16, out_features=2, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=2, out_features=16, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlock(
      (c

In [51]:
S.eval()

ResNetSE(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=16, out_features=2, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=2, out_features=16, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlock(
      (c

In [52]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [53]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [54]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [55]:
len(setfiles)

4708

In [94]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp).cuda()
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)
print(testcount / (endtime-starttime))

0.12144420266151429


In [56]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.Sigmoid'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'models.ResNetBlocks.SELayer'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'models.ResNetBlocks.SEBasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for

In [57]:
flops / 1000000

3461.846256

In [58]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 64, 751]               0
    MelSpectrogram-4              [-1, 64, 751]               0
    InstanceNorm1d-5              [-1, 64, 751]               0
            Conv2d-6          [-1, 16, 64, 751]             160
              ReLU-7          [-1, 16, 64, 751]               0
       BatchNorm2d-8          [-1, 16, 64, 751]              32
            Conv2d-9          [-1, 16, 64, 751]           2,304
             ReLU-10          [-1, 16, 64, 751]               0
      BatchNorm2d-11          [-1, 16, 64, 751]              32
           Conv2d-12          [-1, 16, 64, 751]           2,304
      BatchNorm2d-13          [-1, 16, 64, 751]              32
AdaptiveAvgPool2d-14             [-1, 1

# SE-Resnet34

In [68]:
SpeakerNetModel = importlib.import_module('models.'+'ResNetSE34V2').__getattribute__('MainModel')

In [69]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=64, nOut=192, spec_aug=False, encoder_type='ASP')

ResNetSE34V2.py, Embedding size is 192, Encoder ASP, Spec_aug False.


In [70]:
S.cuda()

ResNetSE(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=32, out_features=4, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=4, out_features=32, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlock(
      (c

In [71]:
S.eval()

ResNetSE(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU(inplace=True)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): SEBasicBlock(
      (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (se): SELayer(
        (avg_pool): AdaptiveAvgPool2d(output_size=1)
        (fc): Sequential(
          (0): Linear(in_features=32, out_features=4, bias=True)
          (1): ReLU(inplace=True)
          (2): Linear(in_features=4, out_features=32, bias=True)
          (3): Sigmoid()
        )
      )
    )
    (1): SEBasicBlock(
      (c

In [72]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [73]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [74]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [75]:
len(setfiles)

4708

In [ ]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)
print(testcount / (endtime-starttime))

In [76]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.Sigmoid'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'models.ResNetBlocks.SELayer'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'models.ResNetBlocks.SEBasicBlock'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for

In [77]:
flops / 1000000

13723.029344

In [78]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 64, 751]               0
    MelSpectrogram-4              [-1, 64, 751]               0
    InstanceNorm1d-5              [-1, 64, 751]               0
            Conv2d-6          [-1, 32, 64, 751]             320
              ReLU-7          [-1, 32, 64, 751]               0
       BatchNorm2d-8          [-1, 32, 64, 751]              64
            Conv2d-9          [-1, 32, 64, 751]           9,216
             ReLU-10          [-1, 32, 64, 751]               0
      BatchNorm2d-11          [-1, 32, 64, 751]              64
           Conv2d-12          [-1, 32, 64, 751]           9,216
      BatchNorm2d-13          [-1, 32, 64, 751]              64
AdaptiveAvgPool2d-14             [-1, 3

# ECAPA-TDNN1024

In [79]:
SpeakerNetModel = importlib.import_module('models.'+'EPACA-TDNN').__getattribute__('MainModel')

In [80]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

EPACA-TDNN.py, Embedding size is 192, Channels 1024, Spec_aug False.


In [81]:
S.cuda()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [82]:
S.eval()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 1024, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(1024, 1024, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(128, 128, kernel_size=(3,), 

In [83]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [84]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [85]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [86]:
len(setfiles)

4708

In [88]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp.cuda()).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)
print(testcount / (endtime-starttime))

0.020084521770477294
248.94792403519492


In [89]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.Spectrogram'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelScale'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelSpectrogram'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[WARN] Cannot find rule for <class 'models.EPACA-TDNN.Conv1dReluBn'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.M

In [90]:
flops / 1000000

13355.652096

In [91]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv1d-6            [-1, 1024, 751]         204,800
       BatchNorm1d-7            [-1, 1024, 751]           2,048
      Conv1dReluBn-8            [-1, 1024, 751]               0
            Conv1d-9            [-1, 1024, 751]       1,048,576
      BatchNorm1d-10            [-1, 1024, 751]           2,048
     Conv1dReluBn-11            [-1, 1024, 751]               0
           Conv1d-12             [-1, 128, 751]          49,152
      BatchNorm1d-13             [-1, 128, 751]             256
           Conv1d-14             [-1, 1

# ECAPA-TDNN512

In [92]:
SpeakerNetModel = importlib.import_module('models.'+'EPACA-TDNNL').__getattribute__('MainModel')

In [93]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

EPACA-TDNNL.py, Embedding size is 192, Channels 512, Spec_aug False.


In [94]:
S.cuda()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 512, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(512, 512, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,)

In [95]:
S.eval()

ECAPA_TDNN(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (layer1): Conv1dReluBn(
    (conv): Conv1d(40, 512, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (layer2): Sequential(
    (0): Conv1dReluBn(
      (conv): Conv1d(512, 512, kernel_size=(1,), stride=(1,), bias=False)
      (bn): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Res2Conv1dReluBn(
      (convs): ModuleList(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (1): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,), bias=False)
        (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,)

In [96]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [97]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [98]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [99]:
len(setfiles)

4708

In [88]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp.cuda()).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)
print(testcount / (endtime-starttime))

0.020084521770477294
248.94792403519492


In [100]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.Spectrogram'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelScale'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelSpectrogram'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[WARN] Cannot find rule for <class 'models.EPACA-TDNNL.Conv1dReluBn'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.

In [101]:
flops / 1000000

3530.241664

In [102]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv1d-6             [-1, 512, 751]         102,400
       BatchNorm1d-7             [-1, 512, 751]           1,024
      Conv1dReluBn-8             [-1, 512, 751]               0
            Conv1d-9             [-1, 512, 751]         262,144
      BatchNorm1d-10             [-1, 512, 751]           1,024
     Conv1dReluBn-11             [-1, 512, 751]               0
           Conv1d-12              [-1, 64, 751]          12,288
      BatchNorm1d-13              [-1, 64, 751]             128
           Conv1d-14              [-1, 

# X-vector

In [103]:
SpeakerNetModel = importlib.import_module('models.'+'X_vector').__getattribute__('MainModel')

In [104]:
# RESNET34SE
# (self, block, layers, num_filters, nOut, spec_aug, encoder_type='SAP', n_mels=40, log_input=True, **kwargs)
# model = ResNetSE(SEBasicBlock, [3, 4, 6, 3], num_filters, nOut, spec_aug, **kwargs)
S = SpeakerNetModel(n_mels=40, nOut=192, spec_aug=False)

X_vector.py, Embedding size is 192,  Spec_aug False.


In [105]:
S.cuda()

Xvector_1L(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (tdnn1): Sequential(
    (0): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn2): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn3): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn4): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU(inplace=

In [106]:
S.eval()

Xvector_1L(
  (instancenorm): InstanceNorm1d(40, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (torchfb): Sequential(
    (0): PreEmphasis()
    (1): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (tdnn1): Sequential(
    (0): Conv1d(40, 512, kernel_size=(5,), stride=(1,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn2): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(2,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn3): Sequential(
    (0): Conv1d(512, 512, kernel_size=(3,), stride=(1,), dilation=(3,))
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (tdnn4): Sequential(
    (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
    (1): ReLU(inplace=

In [107]:
def loadWAV(filename):

    # Maximum audio length
    
    max_audio = int(0*16000 + 240)

    # Read wav file and convert to torch tensor
    sample_rate, audio  = wavfile.read(filename)
    
    audiosize = audio.shape[0]

    if audiosize <= max_audio:
        shortage    = max_audio - audiosize + 1 
        audio       = numpy.pad(audio, (0, shortage), 'wrap')
        audiosize   = audio.shape[0]
    
    feats = []

    feats.append(audio)

    feat = numpy.stack(feats,axis=0).astype(numpy.float)

    return feat

In [108]:
test_list = '/workspace/DATASET/server9_ssd/voxceleb/vox_o_triallist.txt'
test_path = '/workspace/DATASET/server9_ssd/voxceleb'

In [109]:
files = []
with open(test_list) as listfile:
    while True:
        line = listfile.readline()
        if (not line):
            break

        data = line.split();

        ## Append random label if missing
        if len(data) == 2: data = [random.randint(0,1)] + data

        files.append(data[1])
        files.append(data[2])

setfiles = list(set(files))
setfiles.sort()

In [110]:
len(setfiles)

4708

In [58]:
starttime = time.time()
for count, wavline in enumerate(setfiles[:200]):
    wavline = os.path.join(test_path, wavline)
    raw_inp = loadWAV(wavline)
    raw_inp = torch.FloatTensor(raw_inp)
    
    ref_feat = S.forward(raw_inp).detach().cpu().numpy()
    
    print((count+1), end='\r')
endtime =time.time()
print((endtime-starttime)/200)
print(testcount / (endtime-starttime))

0.2526428437232971


In [112]:
flops, params = profile(S, torch.randn(1,1,120000).cuda())

[WARN] Cannot find rule for <class 'torch.nn.modules.instancenorm.InstanceNorm1d'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'utils.PreEmphasis'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.Spectrogram'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelScale'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchaudio.transforms.MelSpectrogram'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[WARN] Cannot find rule for <class 'models.X_vector.AttentiveStatsPool'>. Treat it as zero Macs an

In [113]:
flops / 1000000

2292.281712

In [114]:
summary(S, input_size=(120000,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
       PreEmphasis-1               [-1, 120000]               0
       Spectrogram-2             [-1, 257, 751]               0
          MelScale-3              [-1, 40, 751]               0
    MelSpectrogram-4              [-1, 40, 751]               0
    InstanceNorm1d-5              [-1, 40, 751]               0
            Conv1d-6             [-1, 512, 747]         102,912
              ReLU-7             [-1, 512, 747]               0
       BatchNorm1d-8             [-1, 512, 747]           1,024
            Conv1d-9             [-1, 512, 743]         786,944
             ReLU-10             [-1, 512, 743]               0
      BatchNorm1d-11             [-1, 512, 743]           1,024
           Conv1d-12             [-1, 512, 737]         786,944
             ReLU-13             [-1, 512, 737]               0
      BatchNorm1d-14             [-1, 5